In the same directory, there is `talents.txt` file. That is PDF content provided by Gallup(tm) please note that they are owner of this content.

Countries taken from here: https://gist.github.com/pamelafox/986163 - I had to do some changes, that is why I have a copy.

To run this script pandas, folium, xlrd, xlwt modules are needed.

Have a fun!

Take `talents.txt`, and creates `talents2.xls` with geo coordinates. 

In [1]:
from helper import convert_to_excel

convert_to_excel('talents.txt', 'talents2.xls')

Afghanistan
processing for city
coordinates for Afghanistan->Kabul
https://nominatim.openstreetmap.org/search.php?city=Kabul&country=Afghanistan&format=json
city not located
https://nominatim.openstreetmap.org/search.php?country=Afghanistan&format=json
Argentina
processing for city
coordinates for Argentina->Buenos Aires
https://nominatim.openstreetmap.org/search.php?city=Buenos%20Aires&country=Argentina&format=json
Australia
processing for city
coordinates for Australia->Canberra
https://nominatim.openstreetmap.org/search.php?city=Canberra&country=Australia&format=json
Austria
processing for city
coordinates for Austria->Vienna
https://nominatim.openstreetmap.org/search.php?city=Vienna&country=Austria&format=json
Bahamas
processing for city
coordinates for Bahamas->Nassau
https://nominatim.openstreetmap.org/search.php?city=Nassau&country=Bahamas&format=json
Bahrain
processing for city
coordinates for Bahrain->Manama
https://nominatim.openstreetmap.org/search.php?city=Manama&country=Ba

South Korea
processing for city
coordinates for South Korea->Seoul
https://nominatim.openstreetmap.org/search.php?city=Seoul&country=South%20Korea&format=json
Kuwait
processing for city
coordinates for Kuwait->Kuwait City
https://nominatim.openstreetmap.org/search.php?city=Kuwait%20City&country=Kuwait&format=json
Laos
processing for city
coordinates for Laos->Vientiane
https://nominatim.openstreetmap.org/search.php?city=Vientiane&country=Laos&format=json
Latvia
processing for city
coordinates for Latvia->Riga
https://nominatim.openstreetmap.org/search.php?city=Riga&country=Latvia&format=json
city not located
https://nominatim.openstreetmap.org/search.php?country=Latvia&format=json
Lebanon
processing for city
coordinates for Lebanon->Beirut
https://nominatim.openstreetmap.org/search.php?city=Beirut&country=Lebanon&format=json
Lithuania
processing for city
coordinates for Lithuania->Vilnius
https://nominatim.openstreetmap.org/search.php?city=Vilnius&country=Lithuania&format=json
Luxembou

United Kingdom
processing for city
coordinates for United Kingdom->London
https://nominatim.openstreetmap.org/search.php?city=London&country=United%20Kingdom&format=json
United States
processing for city
coordinates for United States->Washington
https://nominatim.openstreetmap.org/search.php?city=Washington&country=United%20States&format=json
Uruguay
processing for city
coordinates for Uruguay->Montevideo
https://nominatim.openstreetmap.org/search.php?city=Montevideo&country=Uruguay&format=json
United States Virgin Islands
processing for city
coordinates for United States Virgin Islands->Charlotte Amalie
https://nominatim.openstreetmap.org/search.php?city=Charlotte%20Amalie&country=United%20States%20Virgin%20Islands&format=json
city not located
https://nominatim.openstreetmap.org/search.php?country=United%20States%20Virgin%20Islands&format=json
nor country located
Venezuela
processing for city
coordinates for Venezuela->Caracas
https://nominatim.openstreetmap.org/search.php?city=Caraca

Read dataframe from `talents2.xls`

In [2]:
import pandas as pd

df = pd.read_excel('talents2.xls')

remove females, males, all entries (the ones without coordinates)

In [3]:
df = df.dropna(subset=['lon'])
df = df.dropna(subset=['lat'])

Print out map with Learners!

In [53]:
import folium
from folium import plugins


def put_piechart(folium_map, location, row):
    popup_text = f'{index}<br> country: {row["country"]}<br> respondants: {row["respondants"]}'
    if row['respondants'] and row['population']:
        cover =  10000000.0 * float(row['respondants']) / float(row['population'])
    else:
        cover = 1
    if row['respondants']:
        respondants = float(row['respondants']) / 100.0
    else:
        respondants = 1
    if row['population']:
        population = float(row['population']) / 100.0
    else:
        population = 1

    plugins.SemiCircle(location=(row["lat"], row["lon"]), radius=cover, start_angle=0, stop_angle=120,
               color="#007849", fill=True, fill_color="#015627", fill_opacity=0.7,
               popup=popup_text, tooltip="test"
              ).add_to(folium_map)
    plugins.SemiCircle(location=(row["lat"], row["lon"]), radius=respondents, start_angle=120, stop_angle=240,
               color="#8BB0EC", fill=True, fill_color="#6990BA", fill_opacity=0.7,
               popup=popup_text, tooltip="test"
              ).add_to(folium_map)
    plugins.SemiCircle(location=(row["lat"], row["lon"]), radius=radius, start_angle=240, stop_angle=360,
               color="#FF3C3C", fill=True, fill_color="#BB0909", fill_opacity=0.7,
               popup=popup_text, tooltip="test"
              ).add_to(folium_map)


folium_map = folium.Map(location=(52.2297700, 21.0117800), zoom_start=1)

for index, row in df.iterrows():
    put_piechart(folium_map, location=(row["lat"], row["lon"]), row=row)


Help on class CircleMarker in module folium.vector_layers:

class CircleMarker(folium.map.Marker)
 |  CircleMarker(location, radius=10, popup=None, tooltip=None, **kwargs)
 |  
 |  A circle of a fixed size with radius specified in pixels.
 |  
 |  See :func:`folium.vector_layers.path_options` for the `Path` options.
 |  
 |  Parameters
 |  ----------
 |  location: tuple[float, float]
 |      Latitude and Longitude pair (Northing, Easting)
 |  popup: string or folium.Popup, default None
 |      Input text or visualization for object displayed when clicking.
 |  tooltip: str or folium.Tooltip, default None
 |      Display a text when hovering over the object.
 |  radius: float, default 10
 |      Radius of the circle marker, in pixels.
 |  **kwargs
 |      Other valid (possibly inherited) options. See:
 |      https://leafletjs.com/reference-1.6.0.html#circlemarker
 |  
 |  Method resolution order:
 |      CircleMarker
 |      folium.map.Marker
 |      branca.element.MacroElement
 |     

Help on class CircleMarker in module folium.vector_layers:

class CircleMarker(folium.map.Marker)
 |  CircleMarker(location, radius=10, popup=None, tooltip=None, **kwargs)
 |  
 |  A circle of a fixed size with radius specified in pixels.
 |  
 |  See :func:`folium.vector_layers.path_options` for the `Path` options.
 |  
 |  Parameters
 |  ----------
 |  location: tuple[float, float]
 |      Latitude and Longitude pair (Northing, Easting)
 |  popup: string or folium.Popup, default None
 |      Input text or visualization for object displayed when clicking.
 |  tooltip: str or folium.Tooltip, default None
 |      Display a text when hovering over the object.
 |  radius: float, default 10
 |      Radius of the circle marker, in pixels.
 |  **kwargs
 |      Other valid (possibly inherited) options. See:
 |      https://leafletjs.com/reference-1.6.0.html#circlemarker
 |  
 |  Method resolution order:
 |      CircleMarker
 |      folium.map.Marker
 |      branca.element.MacroElement
 |     

Sample output stored in html - github is not supporting folium....

In [54]:
#folium_map.save("sample_learners.html")

Print out output on screen.

In [55]:
folium_map